# Kvikio demo

In [1]:
%load_ext watermark

# These imports are currently unnecessary. I import them to show versions
# cupy_xarray registers the kvikio entrypoint on install.
# import cupy as cp
# import cudf
import cupy_xarray  # registers cupy accessor
import kvikio.zarr

import flox
import numpy_groupies
import numpy as np
import xarray as xr
import zarr

store = "./air-temperature.zarr"

%watermark -iv

flox          : 0.7.3.dev12+g796dcd2
json          : 2.0.9
xarray        : 2023.7.0
cupy_xarray   : 0.1.1+21.gd2da1e4.dirty
kvikio        : 23.2.0
zarr          : 2.16.0
numpy         : 1.24.4
sys           : 3.9.17 | packaged by conda-forge | (main, Aug 10 2023, 07:02:31) 
[GCC 12.3.0]
numpy_groupies: 0.9.22+2.gd148074



In [2]:
xr.backends.list_engines()

{'kvikio': <KvikioBackendEntrypoint>
   Open zarr files (.zarr) using Kvikio
   Learn more at https://docs.rapids.ai/api/kvikio/nightly/api.html#zarr,
 'store': <StoreBackendEntrypoint>
   Open AbstractDataStore instances in Xarray
   Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html,
 'zarr': <ZarrBackendEntrypoint>
   Open zarr files (.zarr) using zarr in Xarray
   Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ZarrBackendEntrypoint.html}

In [23]:
%autoreload

# Consolidated must be False
ds = xr.open_dataset(store, engine="kvikio", consolidated=False)
print(ds.air._variable._data)
ds

MemoryCachedArray(array=CopyOnWriteArray(array=LazilyIndexedArray(array=_ElementwiseFunctionArray(LazilyIndexedArray(array=<cupy_xarray.kvikio.CupyZarrArrayWrapper object at 0x2b5b9f9b4130>, key=BasicIndexer((slice(None, None, None), slice(None, None, None), slice(None, None, None)))), func=functools.partial(<function _scale_offset_decoding at 0x2b5b86161d30>, scale_factor=0.01, add_offset=None, dtype=<class 'numpy.float32'>), dtype=dtype('float32')), key=BasicIndexer((slice(None, None, None), slice(None, None, None), slice(None, None, None))))))


<xarray.Dataset>
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 ...
    scalar   float64 ...
Attributes:
    Conventions:  COARDS
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    title:        4x daily NMC reanalysis (1948)

## Create example dataset

- cannot be compressed

In [ ]:
airt = xr.tutorial.open_dataset("air_temperature", engine="netcdf4")
for var in airt.variables:
    airt[var].encoding["compressor"] = None
airt["scalar"] = 12.0
airt.to_zarr(store, mode="w", consolidated=True)

## Test opening

### Standard usage

In [24]:
xr.open_dataset(store, engine="zarr").air

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
[3869000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature

### Now with kvikio!

 - must read with `consolidated=False` (https://github.com/rapidsai/kvikio/issues/119)
 - dask.from_zarr to GDSStore / open_mfdataset

In [25]:
# Consolidated must be False
ds = xr.open_dataset(store, engine="kvikio", consolidated=False)
print(ds.air._variable._data)
ds

MemoryCachedArray(array=CopyOnWriteArray(array=LazilyIndexedArray(array=_ElementwiseFunctionArray(LazilyIndexedArray(array=<cupy_xarray.kvikio.CupyZarrArrayWrapper object at 0x2b5b9f92e950>, key=BasicIndexer((slice(None, None, None), slice(None, None, None), slice(None, None, None)))), func=functools.partial(<function _scale_offset_decoding at 0x2b5b86161d30>, scale_factor=0.01, add_offset=None, dtype=<class 'numpy.float32'>), dtype=dtype('float32')), key=BasicIndexer((slice(None, None, None), slice(None, None, None), slice(None, None, None))))))


<xarray.Dataset>
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 ...
    scalar   float64 ...
Attributes:
    Conventions:  COARDS
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    title:        4x daily NMC reanalysis (1948)

In [26]:
ds.scalar.variable._data

MemoryCachedArray(array=CopyOnWriteArray(array=LazilyIndexedArray(array=<cupy_xarray.kvikio.CupyZarrArrayWrapper object at 0x2b5b9f9c4310>, key=BasicIndexer(()))))

## Lazy reading

In [27]:
ds.air

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
[3869000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature

## Data load for repr

In [31]:
ds["air"].isel(time=0, lat=10).load()

<xarray.DataArray 'air' (lon: 53)>
array([277.29   , 277.4    , 277.79   , 278.6    , 279.5    , 280.1    ,
       280.6    , 280.9    , 280.79   , 280.69998, 280.79   , 281.     ,
       280.29   , 277.69998, 273.5    , 269.     , 265.5    , 264.     ,
       265.19998, 268.1    , 269.79   , 267.9    , 263.     , 258.1    ,
       254.59999, 251.79999, 249.59999, 249.89   , 252.29999, 254.     ,
       254.29999, 255.89   , 260.     , 263.     , 261.5    , 257.29   ,
       255.5    , 258.29   , 264.     , 268.69998, 270.5    , 270.6    ,
       271.19998, 272.9    , 274.79   , 276.4    , 278.19998, 280.5    ,
       282.9    , 284.69998, 286.1    , 286.9    , 286.6    ],
      dtype=float32)
Coordinates:
    lat      float32 50.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
    time     datetime64[ns] 2013-01-01
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature

In [29]:
ds.scalar

<xarray.DataArray 'scalar' ()>
[1 values with dtype=float64]

## CuPy array on load

In [32]:
type(ds["air"].isel(time=0, lat=10).data)

cupy.ndarray

In [33]:
type(ds["air"].isel(time=0, lat=10).load().data)

cupy.ndarray

## Load to host

In [34]:
ds.air.as_numpy()

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
array([[[241.2    , 242.5    , 243.5    , ..., 232.79999, 235.5    ,
         238.59999],
        [243.79999, 244.5    , 244.7    , ..., 232.79999, 235.29999,
         239.29999],
        [250.     , 249.79999, 248.89   , ..., 233.2    , 236.39   ,
         241.7    ],
        ...,
        [296.6    , 296.19998, 296.4    , ..., 295.4    , 295.1    ,
         294.69998],
        [295.9    , 296.19998, 296.79   , ..., 295.9    , 295.9    ,
         295.19998],
        [296.29   , 296.79   , 297.1    , ..., 296.9    , 296.79   ,
         296.6    ]],

       [[242.09999, 242.7    , 243.09999, ..., 232.     , 233.59999,
         235.79999],
        [243.59999, 244.09999, 244.2    , ..., 231.     , 232.5    ,
         235.7    ],
        [253.2    , 252.89   , 252.09999, ..., 230.79999, 233.39   ,
         238.5    ],
...
        [293.69   , 293.88998, 295.38998, ..., 295.09   , 294.69   ,
         294.29   ],
        [296.29   , 297.19   , 297.59   , ..., 295.29   , 295.09   ,
         294.38998],
        [297.79   , 298.38998, 298.49   , ..., 295.69   , 295.49   ,
         295.19   ]],

       [[245.09   , 244.29   , 243.29   , ..., 241.68999, 241.48999,
         241.79   ],
        [249.89   , 249.29   , 248.39   , ..., 239.59   , 240.29   ,
         241.68999],
        [262.99   , 262.19   , 261.38998, ..., 239.89   , 242.59   ,
         246.29   ],
        ...,
        [293.79   , 293.69   , 295.09   , ..., 295.29   , 295.09   ,
         294.69   ],
        [296.09   , 296.88998, 297.19   , ..., 295.69   , 295.69   ,
         295.19   ],
        [297.69   , 298.09   , 298.09   , ..., 296.49   , 296.19   ,
         295.69   ]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature

In [35]:
type(ds.air.as_numpy().data)

numpy.ndarray

In [36]:
type(ds.air.mean("time").load().data)

cupy.ndarray

## Chunk with dask

In [37]:
ds.chunk(time=10)

<xarray.Dataset>
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(10, 25, 53), meta=np.ndarray>
    scalar   float64 ...
Attributes:
    Conventions:  COARDS
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    title:        4x daily NMC reanalysis (1948)

## GroupBy with flox

Requires

1. flox main branch?
2. https://github.com/ml31415/numpy-groupies/pull/63

In [ ]:
ds.air.groupby("time.month").mean()